# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import os
import datetime
from imageio import imread
import os
from skimage import io
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
from tensorflow.keras import backend as K
tf.random.set_seed(30)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pip install Pillow


In [ ]:
import PIL

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())


In [ ]:
train_path = 'Project_data/train'
val_path = 'Project_data/val'

In [ ]:
image1 = PIL.Image.open(train_path+'/WIN_20180907_15_43_40_Pro_Stop Gesture_new/WIN_20180907_15_43_40_Pro_00011.png')
w,h = image1.size
print(w,h)
image1=imread(train_path+'/WIN_20180907_15_43_40_Pro_Stop Gesture_new/WIN_20180907_15_43_40_Pro_00011.png')
plt.imshow(image1)

In [ ]:
#Verifying cropping output on sample images
image1=imread(train_path+'/WIN_20180907_15_43_40_Pro_Stop Gesture_new/WIN_20180907_15_43_40_Pro_00011.png')
image1 = image1[20:,:-20]
plt.imshow(image1)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
class VideoGenerator:

   # Function for defining image size
    def imageparam(self,image_height,image_width):
        self.image_height=image_height
        self.image_width=image_width

    # Function to define few other parameters
    def othergenparam(self,num_epoches,batch_size,num_images):
        self.num_epoches=num_epoches
        self.batch_size=batch_size
        self.num_images=num_images

    # Generator Function
    def generator(self,source_path,folder_list):
        print( 'Source path = ', source_path, '; batch size =', self.batch_size)
        img_idx = (np.linspace(0,29,self.num_images)).astype(int)
        #create a list of image numbers you want to use for a particular video
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(folder_list)//self.batch_size # calculate the number of batches
            x=len(img_idx)
            y=self.image_width
            z=self.image_height
            for batch in range(num_batches): # we iterate over the number of batches
                batch_data = np.zeros((self.batch_size,x,y,z,3)) 
                #x is the number of images you use for each video,
                # (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((self.batch_size,5)) # batch_labels is the one hot representation of the output
                for folder in range(self.batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*self.batch_size)].split(';')[0]) 
            
                    # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*self.batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
        
                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes
                        
                        #cropping the image
                        #selecting [20:,:-20] matrix as it seemed to be most suitable for maximum images
                    
                        #resizing the image
                        image = image[20:,:-20]
                        image = resize(image,(y,z,3))
                        
                        #batch_data[folder,idx,:,:,0] = image[:,:,0]-(image.mean()) #normalise and feed in the image
                        batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image
                        
                    batch_labels[folder, int(t[folder + (batch*self.batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

            
            # write the code for the remaining data points which are left after full batches
            rem_data = len(folder_list)%self.batch_size
            if (rem_data != 0) :
                batch_size = rem_data
                num_batches = 1
                for batch in range(num_batches): # we iterate over the number of batches
                    batch_data = np.zeros((batch_size,x,y,z,3)) 
                    #x is the number of images you use for each video,
                    # (y,z) is the final size of the input images and 3 is the number of channels RGB
                    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
                    for folder in range(batch_size): # iterate over the batch_size
                        imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
                    
                        # read all the images in the folder
                        for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                            image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                            
                            #crop the images and resize them. Note that the images are of 2 different shape 
                            #and the conv3D will throw error if the inputs in a batch have different shapes

                            #cropping the image
                            #selecting [20:,:-20] matrix as it seemed to be most suitable for maximum images
                            #resizing the image
                            
                            image = image[20:,:-20]
                            image = resize(image,(y,z,3))
                            
                            batch_data[folder,idx,:,:,0] = image[:,:,0]/255 #normalise and feed in the image
                            batch_data[folder,idx,:,:,1] = image[:,:,1]/255 #normalise and feed in the image
                            batch_data[folder,idx,:,:,2] = image[:,:,2]/255 #normalise and feed in the image

                        batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                    yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

    #Function to define parameters for model training           
    def train_model(self,model):

        curr_dt_time = datetime.datetime.now()

        num_train_sequences = len(train_doc)
        print('# training sequences =', num_train_sequences)
        num_val_sequences = len(val_doc)
        print('# validation sequences =', num_val_sequences)
        num_epoches = self.num_epoches # choose the number of epochs
        print ('# epoches =', num_epoches)

        model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)
                
        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
        
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1,min_lr=0.00001) # write the REducelronplateau code here
        #using high factor value here as the validation accuracy is very low compared to train accuracy
        
        callbacks_list = [checkpoint, LR]
        
        #steps per epoch for train data
        if (num_train_sequences%self.batch_size) == 0:
            steps_per_epoch = int(num_train_sequences/self.batch_size)
        else:
            steps_per_epoch = (num_train_sequences//self.batch_size) + 1

        #steps per epoch for val data
        if (num_val_sequences%self.batch_size) == 0:
            validation_steps = int(num_val_sequences/self.batch_size)
        else:
            validation_steps = (num_val_sequences//self.batch_size) + 1

        train_generator = self.generator(train_path,train_doc)
        val_generator = self.generator(val_path,val_doc)

        # Model Fit 
        history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epoches, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
       
        return history

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,LSTM,Dropout,SimpleRNN
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.optimizers import Adam,SGD
from skimage.transform import resize



In [ ]:
class modelbuild3D1(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=64,dropout=0.25):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model
        

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
#Keeping the Batch size = 100 
#keeping the Batch size huge the kernel dies automatically 

In [ ]:
conv_3D1=modelbuild3D1()
conv_3D1.imageparam(image_height=160,image_width=160)
conv_3D1.othergenparam(num_epoches=5,batch_size=100,num_images=15)
conv_3D1_model=conv_3D1.model_definition()
conv_3D1_model.summary()
print("Total Params:", conv_3D1_model.count_params())
model1 = conv_3D1.train_model(conv_3D1_model)

In [ ]:
conv_3D2=modelbuild3D1()
conv_3D2.imageparam(image_height=160,image_width=160)
conv_3D2.othergenparam(num_epoches=5,batch_size=40,num_images=15)
conv_3D2_model=conv_3D2.model_definition()
conv_3D2_model.summary()

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
print("Total Params:", conv_3D2_model.count_params())
model2 = conv_3D2.train_model(conv_3D2_model)

In [ ]:
import matplotlib.pyplot as plt
def plot(history):
    fig, axes = plt.subplots(1,2, figsize=(15,4))

    axes[0].plot(history.history['categorical_accuracy'],label='Training Accuracy')   
    axes[0].plot(history.history['val_categorical_accuracy'],label='Validation Accuracy')
    axes[0].legend(loc='lower right')
    axes[0].title.set_text('Training and Validation Accuracy')

    axes[1].plot(history.history['loss'],label='Training Loss')   
    axes[1].plot(history.history['val_loss'],label='Validation Loss')
    axes[1].legend(loc='upper right')
    axes[1].title.set_text('Training and Validation Loss')

In [ ]:
plot(model2)

In [ ]:
#Changing number of images
conv_3D3=modelbuild3D1()
conv_3D3.imageparam(image_height=160,image_width=160)
conv_3D3.othergenparam(num_epoches=5,batch_size=40,num_images=20)
conv_3D3_model=conv_3D3.model_definition()
conv_3D3_model.summary()
print("Total Params:", conv_3D3_model.count_params())
model3 = conv_3D3.train_model(conv_3D3_model)

In [ ]:
#checking impact of changing image dimensions only
conv_3D4_1=modelbuild3D1()
conv_3D4_1.imageparam(image_height=120,image_width=120)
conv_3D4_1.othergenparam(num_epoches=5,batch_size=40,num_images=15)
conv_3D4_1_model=conv_3D4_1.model_definition()
conv_3D4_1_model.summary()
print("Total Params:", conv_3D4_1_model.count_params())
model4_1 = conv_3D4_1.train_model(conv_3D4_1_model)
plot(model4_1)

In [ ]:
#Changing number of images to 20
conv_3D4=modelbuild3D1()
conv_3D4.imageparam(image_height=120,image_width=120)
conv_3D4.othergenparam(num_epoches=5,batch_size=40,num_images=20)
conv_3D4_model=conv_3D4.model_definition()
conv_3D4_model.summary()
print("Total Params:", conv_3D4_model.count_params())
model4 = conv_3D4.train_model(conv_3D4_model)

In [ ]:
plot(model4)


In [ ]:
#Changing batch size
conv_3D4=modelbuild3D1()
conv_3D4.imageparam(image_height=120,image_width=120)
conv_3D4.othergenparam(num_epoches=5,batch_size=32,num_images=20)
conv_3D4_model=conv_3D4.model_definition()
conv_3D4_model.summary()
print("Total Params:", conv_3D4_model.count_params())
model4 = conv_3D4.train_model(conv_3D4_model)

In [ ]:
plot(model4)

In [ ]:
#num_images=15
conv_3D4=modelbuild3D1()
conv_3D4.imageparam(image_height=120,image_width=120)
conv_3D4.othergenparam(num_epoches=5,batch_size=32,num_images=15)
conv_3D4_model=conv_3D4.model_definition()
conv_3D4_model.summary()
print("Total Params:", conv_3D4_model.count_params())
model4 = conv_3D4.train_model(conv_3D4_model)

In [ ]:
plot(model4)

In [ ]:
class modelbuild3D2(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=128,dropout=0.25):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
#num_images=20,batchsize=40
conv_3D2_1=modelbuild3D2()
conv_3D2_1.imageparam(image_height=120,image_width=120)
conv_3D2_1.othergenparam(num_epoches=5,batch_size=40,num_images=20)
conv_3D2_1_model=conv_3D2_1.model_definition()
conv_3D2_1_model.summary()
print("Total Params:", conv_3D2_1_model.count_params())
model2_1 = conv_3D2_1.train_model(conv_3D2_1_model)

In [ ]:
plot(model2_1)

In [ ]:
#changing image dimensions
conv_3D2_2=modelbuild3D2()
conv_3D2_2.imageparam(image_height=100,image_width=100)
conv_3D2_2.othergenparam(num_epoches=5,batch_size=40,num_images=20)
conv_3D2_2_model=conv_3D2_2.model_definition()
conv_3D2_2_model.summary()
print("Total Params:", conv_3D2_2_model.count_params())
model2_2 = conv_3D2_2.train_model(conv_3D2_2_model)
plot(model2_2)

In [ ]:
# Adding one more layer

In [ ]:
class modelbuild3D3(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=64,dropout=0.25):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Conv3D(128, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
#changing num_images dimensions
conv_3D3_1=modelbuild3D3()
conv_3D3_1.imageparam(image_height=160,image_width=160)
conv_3D3_1.othergenparam(num_epoches=5,batch_size=40,num_images=20)
conv_3D3_1_model=conv_3D3_1.model_definition()
conv_3D3_1_model.summary()
print("Total Params:", conv_3D3_1_model.count_params())
model3_1 = conv_3D3_1.train_model(conv_3D3_1_model)
plot(model3_1)

In [ ]:
#changing num_images dimensions
conv_3D3_3=modelbuild3D3()
conv_3D3_3.imageparam(image_height=120,image_width=120)
conv_3D3_3.othergenparam(num_epoches=5,batch_size=30,num_images=20)
conv_3D3_3_model=conv_3D3_3.model_definition()
conv_3D3_3_model.summary()
print("Total Params:", conv_3D3_3_model.count_params())
model3_3 = conv_3D3_3.train_model(conv_3D3_3_model)
plot(model3_3)

In [ ]:
class modelbuild3D4(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=64,dropout=0.25):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Conv3D(128, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model
        

In [ ]:
#changing num_images dimensions
conv_3D4_1=modelbuild3D4()
conv_3D4_1.imageparam(image_height=120,image_width=120)
conv_3D4_1.othergenparam(num_epoches=5,batch_size=40,num_images=20)
conv_3D4_1_model=conv_3D4_1.model_definition()
conv_3D4_1_model.summary()
print("Total Params:", conv_3D4_1_model.count_params())
model4_1 = conv_3D4_1.train_model(conv_3D4_1_model)
plot(model4_1)

In [ ]:
#changing num_images dimensions
conv_3D4_3=modelbuild3D4()
conv_3D4_3.imageparam(image_height=120,image_width=120)
conv_3D4_3.othergenparam(num_epoches=5,batch_size=25,num_images=20)
conv_3D4_3_model=conv_3D4_3.model_definition()
conv_3D4_3_model.summary()
print("Total Params:", conv_3D4_3_model.count_params())
model4_3 = conv_3D4_3.train_model(conv_3D4_3_model)
plot(model4_3)

In [ ]:
#Changing Dropout
class modelbuild3D5(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=64,dropout=0.5):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Conv3D(128, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
#changing num_images dimensions
conv_3D5_1=modelbuild3D5()
conv_3D5_1.imageparam(image_height=120,image_width=120)
conv_3D5_1.othergenparam(num_epoches=5,batch_size=30,num_images=20)
conv_3D5_1_model=conv_3D5_1.model_definition()
conv_3D5_1_model.summary()
print("Total Params:", conv_3D5_1_model.count_params())
model5_1 = conv_3D5_1.train_model(conv_3D5_1_model)
plot(model5_1)

In [ ]:
#Changing Dropout
class modelbuild3D6(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=64,dropout=0.2):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Conv3D(128, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model
        

In [ ]:
conv_3D6_1=modelbuild3D6()
conv_3D6_1.imageparam(image_height=120,image_width=120)
conv_3D6_1.othergenparam(num_epoches=5,batch_size=30,num_images=20)
conv_3D6_1_model=conv_3D6_1.model_definition()
conv_3D6_1_model.summary()
print("Total Params:", conv_3D6_1_model.count_params())
model6_1 = conv_3D6_1.train_model(conv_3D6_1_model)
plot(model6_1)

In [ ]:
#Changing number of neurons
class modelbuild3D7(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=128,dropout=0.2):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Conv3D(128, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model
        

In [ ]:
conv_3D7_1=modelbuild3D7()
conv_3D7_1.imageparam(image_height=120,image_width=120)
conv_3D7_1.othergenparam(num_epoches=5,batch_size=30,num_images=20)
conv_3D7_1_model=conv_3D7_1.model_definition()
conv_3D7_1_model.summary()
print("Total Params:", conv_3D7_1_model.count_params())
model7_1 = conv_3D7_1.train_model(conv_3D7_1_model)
plot(model7_1)

In [ ]:
#Changing filters to 2,2,2
#Changing number of neurons
class modelbuild3D8(VideoGenerator):
    def model_definition(self,filter=(2,2,2),dense_neurons=64,dropout=0.2):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Conv3D(128, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model
        

In [ ]:
conv_3D8_1=modelbuild3D8()
conv_3D8_1.imageparam(image_height=120,image_width=120)
conv_3D8_1.othergenparam(num_epoches=5,batch_size=30,num_images=20)
conv_3D8_1_model=conv_3D8_1.model_definition()
conv_3D8_1_model.summary()
print("Total Params:", conv_3D8_1_model.count_params())
model8_1 = conv_3D8_1.train_model(conv_3D8_1_model)
plot(model8_1)

In [ ]:
# Changing Optimiser to sgd
class modelbuild3D9(VideoGenerator):
    def model_definition(self,filter=(2,2,2),dense_neurons=64,dropout=0.2):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Conv3D(128, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'sgd'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model
        

In [ ]:
conv_3D9_1=modelbuild3D9()
conv_3D9_1.imageparam(image_height=120,image_width=120)
conv_3D9_1.othergenparam(num_epoches=5,batch_size=40,num_images=20)
conv_3D9_1_model=conv_3D9_1.model_definition()
conv_3D9_1_model.summary()
print("Total Params:", conv_3D9_1_model.count_params())
model9_1 = conv_3D9_1.train_model(conv_3D9_1_model)
plot(model9_1)

In [ ]:
# Changing to 256 neurons 
class modelbuild3D10(VideoGenerator):
    def model_definition(self,filter=(2,2,2),dense_neurons=256,dropout=0.2):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Conv3D(128, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model
        

In [ ]:
# Further Analysing Using a model having 128 neurons by decreasing the batch size further
class modelbuild3D11(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=128,dropout=0.25):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
conv_3D10_1=modelbuild3D10()
conv_3D10_1.imageparam(image_height=120,image_width=120)
conv_3D10_1.othergenparam(num_epoches=5,batch_size=40,num_images=20)
conv_3D10_1_model=conv_3D10_1.model_definition()
conv_3D10_1_model.summary()
print("Total Params:", conv_3D10_1_model.count_params())
model10_1 = conv_3D10_1.train_model(conv_3D10_1_model)
plot(model10_1)

In [ ]:
conv_3D11_1=modelbuild3D11()
conv_3D11_1.imageparam(image_height=120,image_width=120)
conv_3D11_1.othergenparam(num_epoches=5,batch_size=20,num_images=25)
conv_3D11_1_model=conv_3D11_1.model_definition()
conv_3D11_1_model.summary()
print("Total Params:", conv_3D11_1_model.count_params())
model11_1 = conv_3D11_1.train_model(conv_3D11_1_model)
plot(model11_1)

In [ ]:
class modelbuild3D12(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=64,dropout=0.25):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        model.add(Conv3D(16, filter, padding='same',input_shape=input_shape_model))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter, padding='same'))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter, padding='same'))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        #model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
conv_3D12_1=modelbuild3D12()
conv_3D12_1.imageparam(image_height=120,image_width=120)
conv_3D12_1.othergenparam(num_epoches=5,batch_size=20,num_images=20)
conv_3D12_1_model=conv_3D12_1.model_definition(dense_neurons=128,dropout=0.2)
conv_3D12_1_model.summary()
print("Total Params:", conv_3D12_1_model.count_params())
model12_1 = conv_3D12_1.train_model(conv_3D12_1_model)
plot(model12_1)

In [ ]:
conv_3D12_2=modelbuild3D12()
conv_3D12_2.imageparam(image_height=140,image_width=140)
conv_3D12_2.othergenparam(num_epoches=5,batch_size=20,num_images=20)
conv_3D12_2_model=conv_3D12_2.model_definition(dense_neurons=128,dropout=0.2)
conv_3D12_2_model.summary()
print("Total Params:", conv_3D12_2_model.count_params())
model12_2 = conv_3D12_2.train_model(conv_3D12_2_model)
plot(model12_2)

In [ ]:
conv_3D12_4=modelbuild3D12()
conv_3D12_4.imageparam(image_height=120,image_width=120)
conv_3D12_4.othergenparam(num_epoches=5,batch_size=20,num_images=25)
conv_3D12_4_model=conv_3D12_4.model_definition(dense_neurons=128,dropout=0.2)
conv_3D12_4_model.summary()
print("Total Params:", conv_3D12_4_model.count_params())
model12_4 = conv_3D12_4.train_model(conv_3D12_4_model)
plot(model12_4)

In [ ]:
conv_3D12_5=modelbuild3D12()
conv_3D12_5.imageparam(image_height=120,image_width=120)
conv_3D12_5.othergenparam(num_epoches=5,batch_size=15,num_images=25)
conv_3D12_5_model=conv_3D12_5.model_definition(dense_neurons=128,dropout=0.2)
conv_3D12_5_model.summary()
print("Total Params:", conv_3D12_5_model.count_params())
model12_5 = conv_3D12_5.train_model(conv_3D12_5_model)
plot(model12_5)

In [ ]:
# Removing Padding
class modelbuild3D13(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=128,dropout=0.5):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        
        model.add(Conv3D(16, filter,input_shape=input_shape_model))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        #model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
conv_3D13_1=modelbuild3D13()
conv_3D13_1.imageparam(image_height=120,image_width=120)
conv_3D13_1.othergenparam(num_epoches=15,batch_size=20,num_images=23)
conv_3D13_1_model=conv_3D13_1.model_definition()
conv_3D13_1_model.summary()
print("Total Params:", conv_3D13_1_model.count_params())
model13_1 = conv_3D13_1.train_model(conv_3D13_1_model)
plot(model13_1)

In [ ]:
conv_3D13_2=modelbuild3D13()
conv_3D13_2.imageparam(image_height=120,image_width=120)
conv_3D13_2.othergenparam(num_epoches=15,batch_size=20,num_images=23)
conv_3D13_2_model=conv_3D13_2.model_definition(dropout=0.2)
conv_3D13_2_model.summary()
print("Total Params:", conv_3D13_2_model.count_params())
model13_2 = conv_3D13_2.train_model(conv_3D13_2_model)
plot(model13_2)

In [ ]:
conv_3D13_3=modelbuild3D13()
conv_3D13_3.imageparam(image_height=120,image_width=120)
conv_3D13_3.othergenparam(num_epoches=15,batch_size=18,num_images=23)
conv_3D13_3_model=conv_3D13_3.model_definition()
conv_3D13_3_model.summary()
print("Total Params:", conv_3D13_3_model.count_params())
model13_3 = conv_3D13_3.train_model(conv_3D13_3_model)
plot(model13_3)

In [ ]:
conv_3D13_4=modelbuild3D13()
conv_3D13_4.imageparam(image_height=120,image_width=120)
conv_3D13_4.othergenparam(num_epoches=15,batch_size=20,num_images=22)
conv_3D13_4_model=conv_3D13_4.model_definition()
conv_3D13_4_model.summary()
print("Total Params:", conv_3D13_4_model.count_params())
model13_4 = conv_3D13_4.train_model(conv_3D13_4_model)
plot(model13_4)

In [ ]:
conv_3D13_5=modelbuild3D13()
conv_3D13_5.imageparam(image_height=120,image_width=120)
conv_3D13_5.othergenparam(num_epoches=15,batch_size=32,num_images=25)
conv_3D13_5_model=conv_3D13_5.model_definition()
conv_3D13_5_model.summary()
print("Total Params:", conv_3D13_5_model.count_params())
model13_5 = conv_3D13_5.train_model(conv_3D13_5_model)
plot(model13_5)

In [ ]:
# Removing Padding
class modelbuild3D14(VideoGenerator):
    def model_definition(self,filter=(3,3,3),dense_neurons=128,dropout=0.5):
        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        model = Sequential()
        
        model.add(Conv3D(16, filter,input_shape=input_shape_model))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, filter))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, filter))
        model.add(Activation('relu'))
        #model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        model.add(Dropout(dropout))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        #model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(5,activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
conv_3D14_1=modelbuild3D14()
conv_3D14_1.imageparam(image_height=120,image_width=120)
conv_3D14_1.othergenparam(num_epoches=15,batch_size=20,num_images=23)
conv_3D14_1_model=conv_3D14_1.model_definition(dropout=0.2)
conv_3D14_1_model.summary()
print("Total Params:", conv_3D14_1_model.count_params())
model14_1 = conv_3D14_1.train_model(conv_3D14_1_model)
plot(model14_1)

In [ ]:
## 3D RNN

In [ ]:
class Model2DRNN_1(VideoGenerator):

    def model_definition(self,filter=(3,3),dense_neurons=64,dropout=0.25):

        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        
        model = Sequential()
        model.add(TimeDistributed(Conv2D(16, filter , padding='same', activation='relu'),input_shape=input_shape_model))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(32, filter , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(64, filter , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(128, filter , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(256, filter , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))

        model.add(TimeDistributed(Flatten()))


        model.add(LSTM(dense_neurons))
        model.add(Dropout(dropout))
                
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
                
        model.add(Dense(5, activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
pip install --upgrade numpy==1.19.2

In [ ]:
conv_2drnn_1=Model2DRNN_1()
conv_2drnn_1.imageparam(image_height=120,image_width=120)
conv_2drnn_1.othergenparam(num_epoches=5,batch_size=50,num_images=15)
conv_2drnn_1_model=conv_2drnn_1.model_definition()
conv_2drnn_1_model.summary()
print("Total Params:", conv_2drnn_1_model.count_params())
model2d1_1 = conv_2drnn_1.train_model(conv_2drnn_1_model)
plot(model2d1_1)

In [ ]:
conv_2drnn_2=Model2DRNN_1()
conv_2drnn_2.imageparam(image_height=120,image_width=120)
conv_2drnn_2.othergenparam(num_epoches=5,batch_size=20,num_images=15)
conv_2drnn_2_model=conv_2drnn_2.model_definition()
conv_2drnn_2_model.summary()
print("Total Params:", conv_2drnn_2_model.count_params())
model2d1_2 = conv_2drnn_2.train_model(conv_2drnn_2_model)
plot(model2d1_2)

In [ ]:
conv_2drnn_3=Model2DRNN_1()
conv_2drnn_3.imageparam(image_height=120,image_width=120)
conv_2drnn_3.othergenparam(num_epoches=5,batch_size=20,num_images=20)
conv_2drnn_3_model=conv_2drnn_3.model_definition()
conv_2drnn_3_model.summary()
print("Total Params:", conv_2drnn_3_model.count_params())
model2d1_3 = conv_2drnn_3.train_model(conv_2drnn_3_model)
plot(model2d1_3)

In [ ]:
conv_2drnn_4=Model2DRNN_1()
conv_2drnn_4.imageparam(image_height=120,image_width=120)
conv_2drnn_4.othergenparam(num_epoches=5,batch_size=20,num_images=20)
conv_2drnn_4_model=conv_2drnn_4.model_definition(dropout=0.5)
conv_2drnn_4_model.summary()
print("Total Params:", conv_2drnn_4_model.count_params())
model2d1_4 = conv_2drnn_4.train_model(conv_2drnn_4_model)
plot(model2d1_4)

In [ ]:
#GRU
class Model2DRNN_2(VideoGenerator):

    def model_definition(self,filter=(3,3),dense_neurons=64,dropout=0.25):

        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        
        model = Sequential()
        model.add(TimeDistributed(Conv2D(16, filter , padding='same', activation='relu'),input_shape=input_shape_model))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(32, filter , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(64, filter , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(128, filter , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(256, filter , padding='same', activation='relu')))
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))

        model.add(TimeDistributed(Flatten()))

        model.add(GRU(dense_neurons))
        model.add(Dropout(dropout))
                
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
                
        model.add(Dense(5, activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
conv_2drnn_2_1=Model2DRNN_2()
conv_2drnn_2_1.imageparam(image_height=120,image_width=120)
conv_2drnn_2_1.othergenparam(num_epoches=5,batch_size=20,num_images=20)
conv_2drnn_2_1_model=conv_2drnn_2_1.model_definition()
conv_2drnn_2_1_model.summary()
print("Total Params:", conv_2drnn_2_1_model.count_params())
model2d2_1 = conv_2drnn_2_1.train_model(conv_2drnn_2_1_model)
plot(model2d2_1)

In [ ]:
conv_2drnn_2_2=Model2DRNN_2()
conv_2drnn_2_2.imageparam(image_height=120,image_width=120)
conv_2drnn_2_2.othergenparam(num_epoches=5,batch_size=20,num_images=24)
conv_2drnn_2_2_model=conv_2drnn_2_2.model_definition()
conv_2drnn_2_2_model.summary()
print("Total Params:", conv_2drnn_2_2_model.count_params())
model2d2_2 = conv_2drnn_2_2.train_model(conv_2drnn_2_2_model)
plot(model2d2_2)

In [ ]:
#GRU and removing BN
class Model2DRNN_4(VideoGenerator):

    def model_definition(self,filter=(3,3),dense_neurons=64,dropout=0.25):

        input_shape_model = (self.num_images,self.image_width,self.image_height,3)
        
        model = Sequential()
        model.add(TimeDistributed(Conv2D(16, filter , padding='same', activation='relu'),input_shape=input_shape_model))
       # model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(32, filter , padding='same', activation='relu')))
       # model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(64, filter , padding='same', activation='relu')))
       # model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(128, filter , padding='same', activation='relu')))
        #model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
                
        model.add(TimeDistributed(Conv2D(256, filter , padding='same', activation='relu')))
        #model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))

        model.add(TimeDistributed(Flatten()))

        model.add(GRU(dense_neurons))
        model.add(Dropout(dropout))
                
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
                
        model.add(Dense(5, activation='softmax'))

        optimiser = 'adam'
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [ ]:
conv_2drnn_4_1=Model2DRNN_4()
conv_2drnn_4_1.imageparam(image_height=120,image_width=120)
conv_2drnn_4_1.othergenparam(num_epoches=5,batch_size=20,num_images=20)
conv_2drnn_4_1_model=conv_2drnn_4_1.model_definition()
conv_2drnn_4_1_model.summary()
print("Total Params:", conv_2drnn_4_1_model.count_params())
model2d4_1 = conv_2drnn_4_1.train_model(conv_2drnn_4_1_model)
plot(model2d4_1)

In [ ]:
conv_2drnn_4_2=Model2DRNN_4()
conv_2drnn_4_2.imageparam(image_height=120,image_width=120)
conv_2drnn_4_2.othergenparam(num_epoches=15,batch_size=20,num_images=18)
conv_2drnn_4_2_model=conv_2drnn_4_2.model_definition()
conv_2drnn_4_2_model.summary()
print("Total Params:", conv_2drnn_4_2_model.count_params())
model2d4_2 = conv_2drnn_4_2.train_model(conv_2drnn_4_2_model)
plot(model2d4_2)